## Exercise 05.1 (random numbers)

- Using the `randint` function from the `random` module (https://docs.python.org/3/library/random.html#random.randint) to
  develop a function `dice_roll` that emulates the roll of a dice with $n$ sides. The number of sides `n` should an argument to the function.

- For $n=6$, devise and implement a test to check that it is a fair dice.

#### (a) Dice roll code:

In [2]:
from random import randint

def dice_roll(n:int):
    return randint(1, n)

In [3]:
## tests ##
for n in range(1, 20):
    for j in range(100):
        value = dice_roll(n) 
        assert value >= 1 and value <= n

#### (b) Test for fairness

In [9]:
# To check for fairness, we roll the dice a large number of times and check how many times we get each value. 
# We expect to get each value close to $1/6$ of the times. 

n = 6
lst = []

for i in range(0,1000):
    lst.append(dice_roll(n))

items = list(set(lst))
items.sort()

for i in items:
    print("probability of", i, "appearing is", lst.count(i)/len(lst))
    
print("Expected probability =", 1/n)

probability of 1 appearing is 0.187
probability of 2 appearing is 0.173
probability of 3 appearing is 0.161
probability of 4 appearing is 0.159
probability of 5 appearing is 0.18
probability of 6 appearing is 0.14
Expected probability = 0.16666666666666666


## Exercise 05.2 (variance estimation)

For a random variable $X$, the variance of $X$ is defined as  

$$
\begin{align}
\mathrm{Var}\left[ X \right] &= \mathrm{E}\left[\left( X - \mu \right)^2\right]  \\ 
&= \mathrm{E}\left[ X^{2} \right] - \mathrm{E}\left[ X \right]^2
\end{align}
$$

where $\mathrm{E}$ is the 'expectation' (mean of something) and $\mu = \mathrm{E}(X)$ is the mean of $X$. If we have all data (the entire 'population'),
the variance can be computed from:

$$
\mathrm{Var}\left[ X \right] = \frac{\sum_{i=0}^{N-1} x^{2}_{i}}{N} - \left( \frac{ \left( \sum_{i=0}^{N-1} x_{i} \right)}{N} \right)^{2}
$$

Often, we only have a sample of data. For example, we might want to estimate the variance in height for students at a university using just a random sample of students. 
When using a sample from a larger data set to estimate the variance, the above formula has a *bias*. Therefore, it is common to use the *unbiased* estimator

$$
s^{2} = \left( \frac{\sum_{i=0}^{n-1} x^{2}_{i}}{n} - \left( \frac{ \left( \sum_{i=0}^{n-1} x_{i} \right)}{n} \right)^{2} \right) \frac{n}{n-1}
$$

to estimate the variance. In this exercise we will use the unbiased estimator.

1. Create a function that returns the estimated variance for a list of numbers based on the above equation. Test your function using 1 million samples drawn from a Gaussian distribution with a mean of 10 and a standard deviation of 3. Use your crsid to seed the random number generator
   
   *See hint below on how to create the sample.*

2. For a sample drawn from a distribution with mean $5 \times 10^6$ and standard deviation $2.0$, estimate the variance using (i) your function for estimating the variance, and (ii) using the `variance` function from the Python `statistics` module. Comment on and explain any significant differences in the results from (i) and (ii).

### Hint: sampling from a distribution

The function `random.gauss` can be used to sample a Gaussian distribution with a specified mean and standard deviation (square root of the variance) *N* times, e.g.:

In [5]:
import random

random.seed("gnw20")  # See the random number generator

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
x = [random.gauss(mu, sigma) for i in range(8)]


print(x)

[11.176693418313244, 12.08792069683015, 12.649052763696117, 9.845742673456446, 7.317196693264903, 6.876099163343957, 15.181269044169637, 12.844967463974331]


### Solution

1. Estimate variance

In [6]:
from random import gauss
from numpy import array
random.seed("jdb209")

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
n = 10**6

def create_sample(mu, sigma, n):
    return [gauss(mu, sigma) for i in range(n)]

x = create_sample(mu,sigma,n)

def var(x, n:int):
    y = array(x)
    return ((n)/(n-1)) * (sum(y**2)/n - (sum(y)/n)**2)

print("calculated value:",var(x, n))
print("true value:", sigma**2)

calculated value: 8.97772269977105
true value: 9.0


2. Use the `statistics` module to estimate the variance, and compare the estimated variance using `statistics` to the estimated variance using your implementation for the variance estimation.

In [7]:
from statistics import variance
random.seed("jdb209")
mu = 5e6    # mean of the distribution
sigma = 2.0  # Standard deviation
n = 10**6  # number of samples

x = create_sample(mu,sigma,n)
my_value = var(x,n)
package_value = variance(x)

print("My value is:", my_value)
print("The statitistics modules value is", package_value)

My value is: 3.3203158203158205
The statitistics modules value is 3.9900989776779117


My Value of the varience is significantly lower than the varience() function and the true varience of the population in part (ii). Whereas in part (i) the value of the population varience calculated is almost exactly equivelent to that of the true population varience of 9. 

## Exercise 05.03 (optional, parallel processing)

Almost all modern computer processing units have multiple processing *cores*. To utilise the full performance of a processor, operations need to be performed in *parallel*, i.e. each processing core is given a task to perform.

Parallel computing is a very rich and technical area. To help exploit multi-core systems there are libraries that support parallel processing. Below is a simple example using the Python `multiprocessing` library.

In [8]:
# https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
import multiprocess

# On some operaring systems, you may need to uncomment the below line
# multiprocessing.set_start_method('fork')

def f(task):
    """A function that print the input argument and the id for the process that executes the function"""
    print(f"Task index {task}, process id: {multiprocessing.current_process()}\n")
     
    return str(multiprocess.current_process())
        
        
# Using 3 'processes', execute the function 'f' four times (each time with a different argument)  
with multiprocess.Pool(processes=3) as p:
    procs = p.map(f, [0, 1, 2, 3])  # Call function mysort three times
    
    print("Returned data (a list)") 
    print(procs)

NameError: name 'multiprocessing' is not defined

Consider the below code that creates a list of lists of sorted numbers.

In [65]:
def mysort(N):
    """Create a randomly ordered list of integers of length N, and return the sorted list"""
    # Create randomly ordered list
    x = random.sample(range(0, N), N)
 
    # Return sorted list of numbers
    return sorted(x)

def sorted_lists(N, p):
    """Create a list of sorted lists"""
    data = []
    for i in range(p):
        data.append(mysort(N))

%time x = sorted_lists(1000000, 5)

CPU times: user 4.59 s, sys: 162 ms, total: 4.75 s
Wall time: 5.02 s


Use the `multiprocessing` module to perform the above operation in parallel. Investigate how the processing time changes with the number of processes, and in particular the average time per 'create and sort operation' when changing the number of processes.

In [79]:
N = 1000000
p = 5
processes = 3

def task(N):
    sorted_lists(N, p)

with multiprocess.Pool(processes=processes) as p:
    procs = p.map(task,[1000000])

    print(procs)

[None]
